In [1]:
import pandas as pd
import numpy as np

# Users

In [2]:
df_users = pd.read_csv('../data/usuarios.csv')
df_users = df_users.rename(columns={'id':'user_id'})
df_users.head()

,user_id,Occupation,Active Since
0,1,technician,1997-09-22 21:57:58
1,2,other,1998-02-27 03:26:00
2,3,writer,1998-03-07 02:15:39
3,4,technician,1998-04-08 02:09:34
4,5,other,1997-09-30 15:57:44


In [3]:
df_users.Occupation.value_counts()

Occupation
student          193
other            102
educator          94
administrator     77
engineer          67
programmer        65
librarian         51
writer            45
executive         31
scientist         30
technician        27
artist            27
marketing         25
entertainment     17
healthcare        16
ITBA              15
retired           14
lawyer            12
salesman          12
none               9
homemaker          7
doctor             7
Name: count, dtype: int64

In [4]:
# cantidad de ids en el dataframe
len(df_users['user_id'].unique())

943

In [5]:
df_users['Active Since'].iloc[0]

'1997-09-22 21:57:58'

In [6]:
df_users['Active Since'] = df_users['Active Since'].apply(lambda x: pd.to_datetime(x))
df_users['Active Since'].iloc[0]

Timestamp('1997-09-22 21:57:58')

In [7]:
df_users.head(1)

,user_id,Occupation,Active Since
0,1,technician,1997-09-22 21:57:58


# People

In [8]:
df_people = pd.read_csv('../data/personas.csv')
df_people.head(3)

,id,Full Name,year of birth,Gender,Zip Code
0,1,Robert Stanley,1974,M,85711
1,2,Heather Bossey,1945,F,94043
2,3,Don Davis,1975,M,32067


In [9]:
df_people = df_people.rename(columns={'id':'user_id'})
df_people['Zip Code'].value_counts()

Zip Code
55414    9
55105    6
10003    5
20009    5
55337    5
        ..
55038    1
33319    1
97229    1
78209    1
06405    1
Name: count, Length: 795, dtype: int64

In [10]:
# Voy a tirar el zip code, porque no tengo tantos datos para entrenar otro embedding mas
df_people = df_people.drop(columns=['Zip Code'])
df_people.describe()

,user_id,year of birth
count,943.000000,943.000000
mean,472.000000,1963.948038
std,272.364951,12.192740
min,1.000000,1925.000000
25%,236.500000,1955.000000
50%,472.000000,1967.000000
75%,707.500000,1973.000000
max,943.000000,1991.000000


In [11]:
df_people.Gender.value_counts()

Gender
M    670
F    273
Name: count, dtype: int64

In [12]:
df_people.head(1)

,user_id,Full Name,year of birth,Gender
0,1,Robert Stanley,1974,M


# Workers

In [13]:
df_workers = pd.read_csv('../data/trabajadores.csv')
df_workers = df_workers.rename(columns={'id':'user_id'})

df_workers

,user_id,Position,Category,Working Hours,Start Date
0,16,Analyst,B,9 - 18,1998-03-30
1,31,Analyst,A,9 - 18,1997-09-25
2,167,Analyst,B,7 - 16,1997-10-24
3,183,Analyst,C,20 - 04,1998-01-26
4,298,Analyst,C,9 - 18,1997-10-18
5,358,Analyst,A,7 - 16,1998-04-17
6,397,Analyst,B,7 - 16,1998-02-08
7,470,CTO,A,7 - 16,1997-12-23
8,501,CFO,C,7 - 16,1998-01-17
9,598,CEO,A,8-17,1997-12-06


In [14]:
# Esta tabla solo la voy a usar para una variable categorica binaria trabaja/no trabaja
df_workers['Start Date'].iloc[0]

'1998-03-30'

In [15]:
df_workers['Start Date'] = df_workers['Start Date'].apply(lambda x: pd.to_datetime(x))
df_workers['Start Date'].iloc[0]

Timestamp('1998-03-30 00:00:00')

In [16]:
df_workers.head(1)

,user_id,Position,Category,Working Hours,Start Date
0,16,Analyst,B,9 - 18,1998-03-30


# Merge: users + personas + trabajadores

In [17]:
df_merged =df_people.merge(df_users,how='inner',on='user_id')

df_merged.head(3)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58
1,2,Heather Bossey,1945,F,other,1998-02-27 03:26:00
2,3,Don Davis,1975,M,writer,1998-03-07 02:15:39


In [18]:
df_merged.shape,df_merged.dropna().shape,len(df_merged['user_id'].unique())

((943, 6), (943, 6), 943)

In [19]:
df_workers.head(3)

,user_id,Position,Category,Working Hours,Start Date
0,16,Analyst,B,9 - 18,1998-03-30
1,31,Analyst,A,9 - 18,1997-09-25
2,167,Analyst,B,7 - 16,1997-10-24


In [20]:
#Voy a hacer algunos retoques a las columnas antes de continuar

# De la tabla de empleados solo voy a utilizar si trabaja o no trabaja
df_workers = df_workers[['user_id','Category']]
df_workers.head(3)

,user_id,Category
0,16,B
1,31,A
2,167,B


In [21]:
# La union es left porque quiero mantener el total de las personas, no solo aquellas empleadas de la plataforma
df_merged =df_merged.merge(df_workers,how='left',on='user_id')
df_merged.loc[df_merged['Category'].notna(),'Category'] = 1
df_merged['Category']=df_merged['Category'].fillna(0)

df_merged.head(3)

C:\Users\jose_\AppData\Local\Temp\ipykernel_16628\22413777.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged['Category']=df_merged['Category'].fillna(0)


,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Category
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58,0
1,2,Heather Bossey,1945,F,other,1998-02-27 03:26:00,0
2,3,Don Davis,1975,M,writer,1998-03-07 02:15:39,0


In [22]:
df_merged['Category'].value_counts()

Category
0    928
1     15
Name: count, dtype: int64

In [23]:
df_merged = df_merged.rename(columns={'Category':'Trabajador'})
df_merged.head(3)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58,0
1,2,Heather Bossey,1945,F,other,1998-02-27 03:26:00,0
2,3,Don Davis,1975,M,writer,1998-03-07 02:15:39,0


In [24]:
df_merged['Active Since'].min(),df_merged['Active Since'].max()

(Timestamp('1997-09-20 03:05:10'), Timestamp('1998-04-22 23:02:29'))

In [25]:
df_merged['Active Since'].max()-df_merged['Active Since'].min()

Timedelta('214 days 19:57:19')

In [26]:
# Active Since lo voy a cambiar por dias de antiguedad, usando la fecha mas antigua
df_merged['Antiguedad'] = (df_merged['Active Since']-df_merged['Active Since'].min()).dt.days

df_merged.head(3)


,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58,0,2
1,2,Heather Bossey,1945,F,other,1998-02-27 03:26:00,0,160
2,3,Don Davis,1975,M,writer,1998-03-07 02:15:39,0,167


In [27]:
# Check
df_merged['Antiguedad'].min(),df_merged['Antiguedad'].max()

(np.int64(0), np.int64(214))

In [28]:
# Para ver si las vibras astrales tienen correlacion, me voy a quedar con el mes que activo su cuenta
df_merged['Horoscopo Activacion'] = df_merged['Active Since'].dt.month
df_merged.head(3)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58,0,2,9
1,2,Heather Bossey,1945,F,other,1998-02-27 03:26:00,0,160,2
2,3,Don Davis,1975,M,writer,1998-03-07 02:15:39,0,167,3


In [29]:
# Dropeo finalmente las columnas que no me sirven
# df_merged = df_merged.drop(columns=['Full Name','Active Since'])
# df_merged.head(3)

In [30]:
df_merged.shape

(943, 9)

# Ratings

In [31]:
df_ratings = pd.read_csv('../data/scores.csv')
df_ratings = df_ratings.drop(columns=['Unnamed: 0'])

df_ratings.head(3)

,user_id,movie_id,rating,Date
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36


In [32]:
# Check NaNs
df_ratings.shape,df_ratings.dropna().shape,len(df_ratings.user_id.unique())

((100000, 4), (100000, 4), 943)

In [33]:
# Check formto fecha
df_ratings['Date'].iloc[0]

'1997-12-04 15:55:49'

In [34]:
df_ratings['Date'] = df_ratings['Date'].apply(lambda x: pd.to_datetime(x))
df_ratings['Date'].iloc[0]

Timestamp('1997-12-04 15:55:49')

In [35]:
df_ratings.head(3)

,user_id,movie_id,rating,Date
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36


# Movies

In [36]:
df_movies = pd.read_csv('../data/peliculas.csv')

df_movies = df_movies.rename(columns={'id':'movie_id'})
df_movies = df_movies.drop(columns=['IMDB URL'])

df_movies['Release Date'] = df_movies['Release Date'].apply(lambda x: pd.to_datetime(x))


df_movies.head(3)

,movie_id,Name,Release Date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [37]:
df_movies.describe()

,movie_id,Release Date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,1682.000000,1681,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,1989-07-16 12:53:32.373587072,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,...,0.01308,0.014269,0.054697,0.033294,0.036266,0.146849,0.060048,0.149227,0.042212,0.016052
min,1.000000,1922-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,1993-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,1995-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,1996-10-18 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1998-10-23 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,485.695893,NaN,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,...,0.11365,0.118632,0.227455,0.179456,0.187008,0.354061,0.237646,0.356418,0.201131,0.125714


In [38]:
df_movies.shape,df_movies.dropna().shape

((1682, 22), (1681, 22))

In [39]:
df_movies = df_movies[df_movies['unknown']==0]
df_movies = df_movies.drop(columns=['unknown'])

In [40]:
df_movies.shape,df_movies.dropna().shape

((1680, 21), (1680, 21))

# Merged scores +movies

In [41]:
df_ratings.head(1)

,user_id,movie_id,rating,Date
0,196,242,3,1997-12-04 15:55:49


In [42]:
df_ratings = df_ratings.rename(columns={'Date':'Date_rating'})

In [43]:
df_movies.head(1)

,movie_id,Name,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df_movies = df_movies.rename(columns={'Name':'Name_movie'})

In [45]:
# Calculo dias de antiguedad de la peli
df_movies['Antiguedad_pelicula'] = (df_movies['Release Date']-df_movies['Release Date'].min()).dt.days

df_movies.head(3)

,movie_id,Name_movie,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
0,1,Toy Story (1995),1995-01-01,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,26663
1,2,GoldenEye (1995),1995-01-01,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,26663
2,3,Four Rooms (1995),1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,26663


In [46]:
df_movies['Antiguedad_pelicula'].max(),df_movies['Antiguedad_pelicula'].min()

(np.int64(28054), np.int64(0))

In [47]:
# Hago el merge
df_merged_pelis = df_ratings.merge(df_movies,on='movie_id')

df_merged_pelis.head(3)

,user_id,movie_id,rating,Date_rating,Name_movie,Release Date,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
0,196,242,3,1997-12-04 15:55:49,Kolya (1996),1997-01-24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27417
1,186,302,3,1998-04-04 19:22:22,L.A. Confidential (1997),1997-01-01,0,0,0,0,...,1,0,0,1,0,0,1,0,0,27394
2,22,377,1,1997-11-07 07:18:36,Heavyweights (1994),1994-01-01,0,0,0,1,...,0,0,0,0,0,0,0,0,0,26298


In [48]:
df_merged_pelis.shape,df_merged_pelis.dropna().shape

((99990, 25), (99990, 25))

# Merge Final

In [49]:
df_merged.shape,df_merged_pelis.shape

((943, 9), (99990, 25))

In [50]:
df_full = df_merged.merge(df_merged_pelis)
df_full.shape

(99990, 33)

In [51]:
df_full.head(1)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
0,1,Robert Stanley,1974,M,technician,1997-09-22 21:57:58,0,2,9,61,...,0,0,0,0,0,0,0,0,0,26298


In [52]:
df_full.describe()

,user_id,year of birth,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Release Date,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
count,99990.000000,99990.000000,99990,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990,99990,...,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000,99990.000000
mean,462.501120,1965.029593,1997-12-21 02:34:04.862976640,0.012601,91.380708,7.202180,425.534923,3.529893,1997-12-31 00:53:27.417171840,1988-02-09 02:12:40.828082816,...,0.017332,0.053175,0.049545,0.052455,0.194629,0.127313,0.218742,0.093989,0.018542,24145.092139
min,1.000000,1925.000000,1997-09-20 03:05:10,0.000000,0.000000,1.000000,1.000000,1.000000,1997-09-20 03:05:10,1922-01-01 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,254.000000,1958.000000,1997-11-06 22:00:18,0.000000,47.000000,3.000000,175.000000,3.000000,1997-11-13 19:19:19,1986-01-01 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23376.000000
50%,447.000000,1968.000000,1997-12-06 03:05:32,0.000000,77.000000,9.000000,322.000000,4.000000,1997-12-22 21:43:03,1994-01-01 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26298.000000
75%,682.000000,1974.000000,1998-02-14 02:52:00,0.000000,146.000000,11.000000,631.000000,4.000000,1998-02-23 18:53:04,1996-09-28 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27299.000000
max,943.000000,1991.000000,1998-04-22 23:02:29,1.000000,214.000000,12.000000,1682.000000,5.000000,1998-04-22 23:10:38,1998-10-23 00:00:00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,28054.000000
std,266.611967,11.562926,NaN,0.111546,62.853595,4.205607,330.797909,1.125656,NaN,NaN,...,0.130505,0.224384,0.217004,0.222944,0.395917,0.333325,0.413395,0.291815,0.134901,5202.863282


In [53]:
df_full.columns

Index(['user_id', 'Full Name', 'year of birth', 'Gender', 'Occupation',
       'Active Since', 'Trabajador', 'Antiguedad', 'Horoscopo Activacion',
       'movie_id', 'rating', 'Date_rating', 'Name_movie', 'Release Date',
       'Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'Antiguedad_pelicula'],
      dtype='object')

In [54]:
df_full.to_feather('../data/full_data.fth')

In [55]:
# %pip install pyarrow